# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4891, uptime 0:00:17
mini_beamline                    RUNNING   pid 4892, uptime 0:00:17
random_walk                      RUNNING   pid 4893, uptime 0:00:17
random_walk_horiz                RUNNING   pid 4894, uptime 0:00:17
random_walk_vert                 RUNNING   pid 4895, uptime 0:00:17
simple                           RUNNING   pid 4896, uptime 0:00:17
thermo_sim                       RUNNING   pid 4897, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 4898, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 13:20:35
Persistent Unique Scan ID: '51c681ad-2b80-422b-aec0-7b11c37b4725'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 13:20:35.7 |     -1.000 |      93816 |


|         2 | 13:20:35.7 |     -0.500 |      96101 |
|         3 | 13:20:35.7 |      0.000 |      97423 |
|         4 | 13:20:35.8 |      0.500 |      97181 |
|         5 | 13:20:35.8 |      1.000 |      94843 |


+-----------+------------+------------+------------+
generator scan ['51c681ad'] (scan num: 1)





('51c681ad-2b80-422b-aec0-7b11c37b4725',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/51c681ad-2b80-422b-aec0-7b11c37b4725-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 13:20:36
Persistent Unique Scan ID: 'b48d606a-d5a5-42d8-be0b-4dd3591aa7fa'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 13:20:36.2 |          -1 | 1084785347 |


|         2 | 13:20:36.4 |          -0 | 1092163140 |


|         3 | 13:20:36.5 |           0 | 1103911486 |


|         4 | 13:20:36.6 |           0 | 1111676643 |


|         5 | 13:20:36.7 |           1 | 1116678649 |


+-----------+------------+-------------+------------+
generator scan ['b48d606a'] (scan num: 2)





('b48d606a-d5a5-42d8-be0b-4dd3591aa7fa',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 13:20:37
Persistent Unique Scan ID: 'f73441f0-919d-4d23-97a5-9e0d43edc59c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 13:20:37.2 | 1131570905 |


|         2 | 13:20:38.2 | 1096508446 |


|         3 | 13:20:39.2 | 1053413650 |


...

|        28 | 13:21:04.2 | 1091478827 |


|        29 | 13:21:05.2 | 1131266517 |


|        30 | 13:21:06.2 | 1094519129 |


+-----------+------------+------------+
generator count ['f73441f0'] (scan num: 3)





('f73441f0-919d-4d23-97a5-9e0d43edc59c',)